In [5]:
!pip install langchain langchain-google-genai sentence-transformers pymongo datasets langchain-community
!pip install -q accelerate
!pip install pymongo[srv]

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.


In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from pymongo import MongoClient

In [2]:
import pymongo
import pandas as pd
from google.colab import userdata

# Step 1: Connect to MongoDB
db_url = userdata.get("phongtro123_db_url")
client = pymongo.MongoClient(db_url)  # Update with your MongoDB connection string
db = client["phongtro123"]  # Replace with your database name
collection = db["embedding_for_vector_search"]  # Repla

In [3]:
data = list(collection.find())
df = pd.DataFrame(data)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              2000 non-null   object
 1   title            2000 non-null   object
 2   price            2000 non-null   object
 3   address          2000 non-null   object
 4   area             2000 non-null   object
 5   description      2000 non-null   object
 6   contact_name     2000 non-null   object
 7   phone            2000 non-null   object
 8   post_id          2000 non-null   int64 
 9   url              2000 non-null   object
 10  type             2000 non-null   object
 11  embedding_title  2000 non-null   object
dtypes: int64(1), object(11)
memory usage: 187.6+ KB


In [5]:
embedding = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")

/tmp/ipython-input-3199800100.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

bpe.codes: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [47]:
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embedding,
    index_name="vector_index",
    text_key="title",  # hoặc description
    embedding_key="embedding_title"
)

In [48]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.7,
    google_api_key=userdata.get("api_key")
)

In [50]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # hoặc "map_reduce" nếu muốn tóm tắt nhiều kết quả
    verbose=True
)

In [51]:
query = "Tìm phòng trọ gần đại học Bách Khoa"
answer = rag_chain.run(query)
print(answer)



> Entering new RetrievalQA chain...

> Finished chain.
Dựa trên thông tin bạn cung cấp, có các phòng trọ cho thuê gần Đại học Bách Khoa:

*   Cho thuê phòng giá rẻ Gần Đại Học Bách Khoa.
*   Phòng trọ giá rẻ gần ĐH Bách Khoa Q10 có gác đầy đủ nội thất.


In [52]:
!pip install fastapi uvicorn pyngrok

In [54]:
from pyngrok import ngrok

# Dán token của bạn vào đây 👇
!ngrok config add-authtoken 3594P35osJq5y4Of2CUu38n1WDz_2frNdf8MCuZn1gFvjwemA

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [63]:
from fastapi import FastAPI, Query
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# --- Import mô hình RAG của bạn ---
# giả sử bạn đã load sẵn các biến `rag_chain`

app = FastAPI(title="RAG API with Gemini")

class QueryRequest(BaseModel):
    question: str

@app.post("/ask")
async def ask_rag(request: QueryRequest):
    result = rag_chain.invoke(request.question)
    return {
        "query": request.question,
        "answer": result["result"]
    }


In [65]:
import nest_asyncio
import uvicorn
import asyncio
from pyngrok import ngrok

# Cho phép event loop lồng nhau trong Colab
nest_asyncio.apply()

# Mở cổng public bằng ngrok
public_url = ngrok.connect(8000)
print(f"🚀 Public API URL: {public_url.public_url}/ask")

# Tạo cấu hình và chạy server
config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

# Chạy server bất đồng bộ (Colab-friendly)
await server.serve()

🚀 Public API URL: https://cribla-jonas-overclosely.ngrok-free.dev/ask


INFO:     Started server process [4096]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)




> Entering new RetrievalQA chain...

> Finished chain.
INFO:     2405:4803:c663:9800:842e:45c5:1acc:d379:0 - "POST /ask HTTP/1.1" 200 OK


> Entering new RetrievalQA chain...

> Finished chain.
INFO:     2405:4803:c663:9800:842e:45c5:1acc:d379:0 - "POST /ask HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4096]


In [66]:
import requests

res = requests.post(
    "https://cribla-jonas-overclosely.ngrok-free.dev/ask",
    json={"question": "Tìm phòng trọ gần Đại học Bách Khoa giá dưới 6 triệu"}
)
print(res.json())


InvalidURL: Failed to parse: https://<thay_bằng_ngrok_URL>/ask